In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors
import sys
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Path
os.getcwd()

'/Volumes/GoogleDrive/Shared drives/Lauren and Hilary/Regional Overland Flow Project/Data and Scripts/Regional-OF'

In [10]:
os.chdir('../San Pedro River/DEMs/')
sys.path.insert(0, '/Volumes/GoogleDrive/Shared drives/Lauren and Hilary/Regional Overland Flow Project/Data and Scripts/pysheds_trial/pysheds-master')
from pysheds.grid import Grid

In [11]:
path = Path('./')
Path('output').mkdir(parents=True, exist_ok=True)
output_path = 'output/mosaic_output.tif'

In [12]:
raster_files = list(path.iterdir())
raster_to_mosiac = []

In [13]:
input_file = '../n30w120_dem/n30w120_dem.tif'

In [14]:
try:
    grid = Grid.from_raster(input_file)
except TypeError:
    grid = Grid.from_raster(input_file, nodata = -9999)
dem = grid.read_raster(input_file, nodata=grid.nodata)

In [15]:
# Condition DEM
# ----------------------
# Fill pits in DEM
pit_filled_dem = grid.fill_pits(dem)

# Fill depressions in DEM
flooded_dem = grid.fill_depressions(pit_filled_dem)
    
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)

In [ ]:
# Determine D8 flow directions from DEM
# ----------------------
# Specify directional mapping
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    
# Compute flow directions
# -------------------------------------
fdir = grid.flowdir(inflated_dem, dirmap=dirmap)

In [ ]:
fig = plt.figure(figsize=(8,6))
fig.patch.set_alpha(0)

plt.imshow(fdir, extent=grid.extent, cmap='viridis', zorder=2)
boundaries = ([0] + sorted(list(dirmap)))
plt.colorbar(boundaries= boundaries,
             values=sorted(dirmap))
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Flow direction grid', size=14)
plt.grid(zorder=-1)
plt.tight_layout()